<a href="https://colab.research.google.com/github/naveenkumark1/Extensive_Vision_and_AI_V.4/blob/master/Week2/pytorch.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [0]:
from __future__ import print_function # Directive to the compiler to use "print" statement as per one of the future versions of python untill it becomes a standard
import torch                          # Importing the pytorch library module
import torch.nn as nn                 # Importing the torch.nn module to use its functions to build a neural network - Mostly object oriented 
import torch.nn.functional as F       # Importing the "functional" function from the torch.nn module  - Mostly functional consists of functions ( to just run a specific function )
import torch.optim as optim           # Importing the torch.optim module to use its functions to optimise our neural network
from torchvision import datasets, transforms  # Importing the data sets and the transforms module from the torchvision library

# datasets : contatins the image data sets ex. MNIST
# transforms : contains the image tranforming functions ex. PIL to other forms, Croping, Padding, Resizing etc

In [0]:
class Net(nn.Module):                               ## Creating a class for the Neural Network
    def __init__(self):                             ## Initialising a Self module 
        super(Net, self).__init__()                 ## Initialisation to inherit the methods and (attributes?) from the parent class  -- Explore OOP  -- Super init error

        ## Defining the Convolution Neural Network using Conv2d form the nn module 
        ## nn.Conv2d(in_channels, out_channels, kernel_size, stride=1, padding=0, dilation=1, groups=1, bias=True, padding_mode='zeros')
        ## Mentioned only local receptive for each kernel - 3X3

        self.conv1 = nn.Conv2d(1, 32, 3, padding=1)  #1 Layer (in- 1 channel-Image)(out-32 feature channels result of convolution)(equal kernal-3X3)(padding-1pixel around the image)
        # Layer 1 - RF - Local - 3X3 & Global - 3X3

        self.conv2 = nn.Conv2d(32, 64, 3, padding=1) #2 Layer (in-32 channel features)(out-64 feature channels result of convolution)(equal kernal-3X3)(padding-1pixel around the image)
        # Layer 2 - RF - Local - 3X3 & Global - 5X5

        ## Defining a max pooling layer using MaxPool2d form the nn module 
        ## nn.MaxPool2d(kernel_size, stride=None, padding=0, dilation=1, return_indices=False, ceil_mode=False)
        self.pool1 = nn.MaxPool2d(2, 2)              #3 layer (equal max pooling layer 2X2)(stride-2)
        # Layer Pool 1 - Doubles the Global receptive field to 10X10

        self.conv3 = nn.Conv2d(64, 128, 3, padding=1)#4 Layer (in-64 channel features)(out-128 feature channels result of convolution)(equal kernal-3X3)(padding-1pixel around the image)
        # Layer 3 - RF - Local - 3X3 & Global - 10X10

        self.conv4 = nn.Conv2d(128, 256, 3, padding=1)#5 Layer (in-128 channel features)(out-256 feature channels result of convolution)(equal kernal-3X3)(padding-1pixel around the image)
        # Layer 4 - RF - Local - 3X3 & Global - 12X12
        
        self.pool2 = nn.MaxPool2d(2, 2)              #6 layer (equal pooling layer 2X2)(stride-2)
        # Layer Pool 2 - Doubles the Global receptive field to 24X24

        self.conv5 = nn.Conv2d(256, 512, 3)          #7 Layer (in-256 channel features)(out-512 feature channels result of convolution)(equal kernal-3X3)(padding-1pixel around the image)
        # Layer 5 - RF - Local - 3X3 & Global - 24X24

        self.conv6 = nn.Conv2d(512, 1024, 3)         #8 Layer (in-512 channel features)(out-1024 feature channels result of convolution)(equal kernal-3X3)(padding-1pixel around the image)
        # Layer 6 - RF - Local - 3X3 & Global - 26X26

        ## In the last layer we convolve all the feature layers in to the 10 output classes of MNIST ( 0 to 9)

        self.conv7 = nn.Conv2d(1024, 10, 3)          #9 Layer (in-1024 channel features)(out-10 Classes of output)(equal kernal-3X3)(padding-1pixel around the image)  
        # Layer 7 - RF - Local - 3X3 & Global - 28X28

# Above we defined a network structure, Now we will defing the route for the informaiton to travel through the network in a single direction ( forward direction )

    def forward(self, x):  ## Define a forward path for 'self' with input 'x'
        x = self.pool1(F.relu(self.conv2(F.relu(self.conv1(x)))))   # input x to "conv1" layer -> send through "relu" activation -> convolve with "conv2" -> send through "relu" activation -> max pool at "pool1" layer
        x = self.pool2(F.relu(self.conv4(F.relu(self.conv3(x)))))   # input the output of pool1 to "conv3" layer -> send through "relu" activation -> convolve with "conv4" -> send through "relu" activation -> max pool at "pool2" layer
        x = F.relu(self.conv6(F.relu(self.conv5(x))))               # input the output of pool2 to "conv5" layer -> send through "relu" activaiton -> convolve with "conv6" -> send through "relu" activation
        x = F.relu(self.conv7(x))                                   # input the output of "relu" activated "conv6" -> to "conv7" -> send through "relu" activation -> fires one of the 10 classes of output of MNIST
        x = x.view(-1, 10)                                          # View one of the 10 classes that is fired in the above layer
        return F.log_softmax(x)                                     # To get the probability of the output we use softmax

#Global Receptive Field Summary 

###Layer 1 : 3X3 
###Layer 2 : 5X5
###Layer 3 : MAX POOL - RFX2 - 10X10
###Layer 4 : 10X10
###Layer 5 : 12X12
###Layer 6 : MAX POOL - RFX2 - 24X24
###Layer 7 : 24X24
###Layer 8 : 26X26
###Layer 9 : 28X28 - Size of the Image



In [3]:
## This block prints the summary of the above Neural network

#!pip install torchsummary
from torchsummary import summary
use_cuda = torch.cuda.is_available()                   # Selecting between a GPU and CPU based on Availability
device = torch.device("cuda" if use_cuda else "cpu")   # Selecting between a GPU and CPU based on Availability
model = Net().to(device)                               # Selecting between a GPU and CPU based on Availability
summary(model, input_size=(1, 28, 28))                 # Show the summary fo the "model"(Net) where input dimension is (1X28X28)

----------------------------------------------------------------
        Layer (type)               Output Shape         Param #
            Conv2d-1           [-1, 32, 28, 28]             320
            Conv2d-2           [-1, 64, 28, 28]          18,496
         MaxPool2d-3           [-1, 64, 14, 14]               0
            Conv2d-4          [-1, 128, 14, 14]          73,856
            Conv2d-5          [-1, 256, 14, 14]         295,168
         MaxPool2d-6            [-1, 256, 7, 7]               0
            Conv2d-7            [-1, 512, 5, 5]       1,180,160
            Conv2d-8           [-1, 1024, 3, 3]       4,719,616
            Conv2d-9             [-1, 10, 1, 1]          92,170
Total params: 6,379,786
Trainable params: 6,379,786
Non-trainable params: 0
----------------------------------------------------------------
Input size (MB): 0.00
Forward/backward pass size (MB): 1.51
Params size (MB): 24.34
Estimated Total Size (MB): 25.85
-------------------------------------

/usr/local/lib/python3.6/dist-packages/ipykernel_launcher.py:48: UserWarning: Implicit dimension choice for log_softmax has been deprecated. Change the call to include dim=X as an argument.


In [4]:
torch.manual_seed(1)                                                       # Set the seed to enure the repeatability of the data 
batch_size = 128                                                           # Number of items to be passed in the network in one iteration

kwargs = {'num_workers': 1, 'pin_memory': True} if use_cuda else {}        #
train_loader = torch.utils.data.DataLoader(                                # Use the torch.util.DataLoder module to load the MNIST data from internet in to the train_loader variable 
    datasets.MNIST('../data', train=True, download=True,                   # Data goes in to the training data set and the data will be downloaded in to the machine
                    transform=transforms.Compose([                         # Use the transforms.compose module to transform the input images
                        transforms.ToTensor(),                             # The input data is converted in to tensors compatible with pytorch
                        transforms.Normalize((0.1307,), (0.3081,))         # normalise the data using the mean and the standard deviation of the MNIST data
                    ])),
    batch_size=batch_size, shuffle=True, **kwargs)                         # Setting batch size to 128 & selecting random data by shuffling

test_loader = torch.utils.data.DataLoader(                                 # SIMILAR PROCESS FOR THE TEST DATA ALSO
    datasets.MNIST('../data', train=False, transform=transforms.Compose([
                        transforms.ToTensor(),
                        transforms.Normalize((0.1307,), (0.3081,))
                    ])),
    batch_size=batch_size, shuffle=True, **kwargs)

# Explore - Mean, Stdev, Kwargs

  0%|          | 0/9912422 [00:00<?, ?it/s]

9920512it [00:00, 26535509.91it/s]                            


Extracting ../data/MNIST/raw/train-images-idx3-ubyte.gz to ../data/MNIST/raw


32768it [00:00, 283147.51it/s]                           
0it [00:00, ?it/s]

Extracting ../data/MNIST/raw/train-labels-idx1-ubyte.gz to ../data/MNIST/raw


1654784it [00:00, 7533877.10it/s]                            
8192it [00:00, 177155.89it/s]

Extracting ../data/MNIST/raw/t10k-images-idx3-ubyte.gz to ../data/MNIST/raw
Extracting ../data/MNIST/raw/t10k-labels-idx1-ubyte.gz to ../data/MNIST/raw
Processing...
Done!


In [0]:
from tqdm import tqdm                                           # Including the taquadam library for the progress bar and has good support for nested loops
def train(model, device, train_loader, optimizer, epoch):       # Define a function "train" with (model, device, train_loader, optimizer, epoch) arguments
    model.train()                                               # Creating a funciton called model.train()
    pbar = tqdm(train_loader)                                   # creating a progress bar using train_loader
    for batch_idx, (data, target) in enumerate(pbar):           # Initialising a loop for the network taining
        data, target = data.to(device), target.to(device)       # Initialising the images and their corresponding labels to variables
        optimizer.zero_grad()                                   # Resetting the values of the optimizer to zero before starting 
        output = model(data)                                    # "output" is the output of the "model" form "data" - Training in this case
        loss = F.nll_loss(output, target)                       # Comparing the output of the NN and the actual to find the loss
        loss.backward()                                         # Backward propogation of the loss
        optimizer.step()                                        # Using the optimizer to update the weights as per the leanring rate
        pbar.set_description(desc= f'loss={loss.item()} batch_id={batch_idx}') # Displaying the loss or a corresponding batch along with the descreption bar


def test(model, device, test_loader):                           # Create a function test with (model, device, test_loader) as arguments
    model.eval()                                                # Creating a funciton called model.eval()
    test_loss = 0                                               # Initialising the test loss to Zero
    correct = 0                                                 # Initialising the correct to Zero
    with torch.no_grad():                                       # Set all grad flags to Zero
        for data, target in test_loader:                        # Start the loop for data and target in the test_loader
            data, target = data.to(device), target.to(device)   # Initialising the images and their corresponding labels to variables
            output = model(data)                                # "output" is the output of the "model" form "data" - Testing in this case
            test_loss += F.nll_loss(output, target, reduction='sum').item()  # sum up batch loss  - Loss of the complete batch
            pred = output.argmax(dim=1, keepdim=True)           # get the index of the max log-probability - ??
            correct += pred.eq(target.view_as(pred)).sum().item()# Counting the correct results given by the model

    test_loss /= len(test_loader.dataset)                       # Finding the percent of test loss

    print('\nTest set: Average loss: {:.4f}, Accuracy: {}/{} ({:.0f}%)\n'.format(
        test_loss, correct, len(test_loader.dataset),           # Printing the Testloss, Correct values & the percentage of the loss
        100. * correct / len(test_loader.dataset)))

In [6]:

model = Net().to(device)                                         # Initialising the "model" with the "Net" neural network   
optimizer = optim.SGD(model.parameters(), lr=0.01, momentum=0.9) # Using the stochastic gradient descent with leanring rate 0.01 & momentum ??

for epoch in range(1, 2):                                        # Starting the loop with epochs
    train(model, device, train_loader, optimizer, epoch)         # running the train module defined above
    test(model, device, test_loader)                             # running the test module defined above

loss=1.7953499555587769 batch_id=468: 100%|██████████| 469/469 [00:17<00:00, 26.36it/s]



Test set: Average loss: 1.7226, Accuracy: 3784/10000 (38%)

